In [1]:
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
my_url = "https://www.uscourts.gov/judges-judgeships/judicial-vacancies/judicial-emergencies"
raw_html = requests.get(my_url).content
soup_doc = BeautifulSoup(raw_html, "html.parser")
print(type(soup_doc))

<class 'bs4.BeautifulSoup'>


In [3]:
### TEST VIEWS OF DATA
# raw_html
# print(soup_doc)
print(soup_doc.prettify())

<!DOCTYPE html>
<!--[if IEMobile 7]><html class="iem7"  lang="en" dir="ltr"><![endif]-->
<!--[if lte IE 6]><html class="lt-ie9 lt-ie8 lt-ie7"  lang="en" dir="ltr"><![endif]-->
<!--[if (IE 7)&(!IEMobile)]><html class="lt-ie9 lt-ie8"  lang="en" dir="ltr"><![endif]-->
<!--[if IE 8]><html class="lt-ie9"  lang="en" dir="ltr"><![endif]-->
<!--[if (gte IE 9)|(gt IEMobile 7)]><!-->
<html dir="ltr" lang="en" prefix="og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product#">
 <!--<![endif]-->
 <head>
  <!--[if IE]><![endif]-->
  <link href="//www.google-analytics.com" rel="dns-prefetch"/>
  <link href="https://www.google-analytics.com" rel="preconnect"/>
  <link href="//stats.g.doubleclick.net" rel="dns-prefetch"/>
  <link href="https://stats.g.doubleclick.net" rel="preconnect"/>
  <link href="//ws.sharethis.com" rel="dns-prefetch"/>
  <link href="https://ws.sharethis

In [4]:
# soup_doc.find_all('tbody')[0].find_all('tr')[0]

In [5]:
all_rows = soup_doc.find_all('tbody')[0].find_all('tr')
emrg_table = []
for row in all_rows:
    emrg_seat = {}
    emrg_seat['Court_Code'] = row.find_all(attrs={"data-th":"Court:"})[0].string
    emrg_seat['Vacancy_Created_By'] = row.find_all(attrs={"data-th":"Vacancy Created By:"})[0].string
    emrg_seat['Vacancy_Reason'] = row.find_all(attrs={"data-th":"Reason:"})[0].string
    emrg_seat['Vacancy_Date'] = row.find_all(attrs={"data-th":"Vacancy Date:"})[0].string
    emrg_seat['Days_Pending_AT_SCRAPE'] = row.find_all(attrs={"data-th":"Days Pending:"})[0].string
    emrg_seat['Weighted_Filings_Per_Seat'] = row.find_all(attrs={"data-th":"Weighted Filings per Judgeship *:"})[0].string
    emrg_seat['Adjusted_Filings_Per_Panel'] = row.find_all(attrs={"data-th":"Adjusted Filings per Panel *:"})[0].string
    emrg_table.append(emrg_seat)
emrg_table

[{'Court_Code': '02 - NY-E',
  'Vacancy_Created_By': 'Feuerstein,Sandra J.',
  'Vacancy_Reason': 'Senior',
  'Vacancy_Date': '01/21/2015',
  'Days_Pending_AT_SCRAPE': '1626',
  'Weighted_Filings_Per_Seat': '559',
  'Adjusted_Filings_Per_Panel': '0'},
 {'Court_Code': '02 - NY-E',
  'Vacancy_Created_By': 'Vitaliano,Eric N.',
  'Vacancy_Reason': 'Senior',
  'Vacancy_Date': '02/28/2017',
  'Days_Pending_AT_SCRAPE': '857',
  'Weighted_Filings_Per_Seat': '559',
  'Adjusted_Filings_Per_Panel': '0'},
 {'Court_Code': '02 - NY-E',
  'Vacancy_Created_By': 'Amon,Carol Bagley',
  'Vacancy_Reason': 'Senior',
  'Vacancy_Date': '11/30/2016',
  'Days_Pending_AT_SCRAPE': '947',
  'Weighted_Filings_Per_Seat': '559',
  'Adjusted_Filings_Per_Panel': '0'},
 {'Court_Code': '02 - NY-E',
  'Vacancy_Created_By': 'Gleeson,John',
  'Vacancy_Reason': 'Resigned',
  'Vacancy_Date': '03/09/2016',
  'Days_Pending_AT_SCRAPE': '1213',
  'Weighted_Filings_Per_Seat': '559',
  'Adjusted_Filings_Per_Panel': '0'},
 {'Court_C

In [6]:
df = pd.DataFrame(emrg_table)

In [7]:
df['tempFirst'] = df.Vacancy_Created_By.str.extract(r',(.*)')
df['tempLast'] = df.Vacancy_Created_By.str.extract(r'(.*),')
df['First_Name'] = df.tempFirst.str.extract(r'(^\w+\.*)\s*')
df['Middle_Name'] = df.tempFirst.str.extract(r'\s(\w*\.*)$')
df['Last_Name'] = df.tempLast.str.extract(r'(^\w+)\s*')
df['Suffix'] = df.tempLast.str.extract(r'\s(\w*\.*)$')
df['Vacancy_Created_By'] = df.First_Name + ' ' + df.Middle_Name.fillna('') + ' ' + df.Last_Name + ', ' + df.Suffix.fillna('')
df.Vacancy_Created_By.replace(r'\s{2,4}',' ',regex=True,inplace=True)
df.Vacancy_Created_By.replace(r', $','',regex=True,inplace=True)
df = df.drop(['tempFirst','tempLast'], axis=1)

In [8]:
df = df[['Court_Code','Vacancy_Created_By','Vacancy_Reason','Vacancy_Date','Days_Pending_AT_SCRAPE','Weighted_Filings_Per_Seat','Adjusted_Filings_Per_Panel','First_Name','Middle_Name','Last_Name','Suffix']]
df.head(60)

,Court_Code,Vacancy_Created_By,Vacancy_Reason,Vacancy_Date,Days_Pending_AT_SCRAPE,Weighted_Filings_Per_Seat,Adjusted_Filings_Per_Panel,First_Name,Middle_Name,Last_Name,Suffix
0,02 - NY-E,Sandra J. Feuerstein,Senior,01/21/2015,1626,559,0,Sandra,J.,Feuerstein,NaN
1,02 - NY-E,Eric N. Vitaliano,Senior,02/28/2017,857,559,0,Eric,N.,Vitaliano,NaN
2,02 - NY-E,Carol Bagley Amon,Senior,11/30/2016,947,559,0,Carol,Bagley,Amon,NaN
3,02 - NY-E,John Gleeson,Resigned,03/09/2016,1213,559,0,John,NaN,Gleeson,NaN
4,02 - NY-E,Joseph F. Bianco,Elevated,05/08/2019,58,559,0,Joseph,F.,Bianco,NaN
5,02 - NY-S,P. Kevin Castel,Senior,08/05/2017,699,575,0,P.,Kevin,Castel,NaN
6,02 - NY-S,Loretta A. Preska,Senior,03/01/2017,856,575,0,Loretta,A.,Preska,NaN
7,02 - NY-S,Paul A. Crotty,Senior,08/01/2015,1434,575,0,Paul,A.,Crotty,NaN
8,02 - NY-W,William M. Skretny,Senior,03/08/2015,1580,592,0,William,M.,Skretny,NaN
9,03 - NJ,Robert B. Kugler,Senior,11/02/2018,245,903,0,Robert,B.,Kugler,NaN


In [9]:
forceInt = ['Days_Pending_AT_SCRAPE','Weighted_Filings_Per_Seat','Adjusted_Filings_Per_Panel']
df[forceInt] = df[forceInt].astype('int')
df.Vacancy_Date = df.Vacancy_Date.apply(pd.to_datetime)
df.dtypes

Court_Code                            object
Vacancy_Created_By                    object
Vacancy_Reason                        object
Vacancy_Date                  datetime64[ns]
Days_Pending_AT_SCRAPE                 int32
Weighted_Filings_Per_Seat              int32
Adjusted_Filings_Per_Panel             int32
First_Name                            object
Middle_Name                           object
Last_Name                             object
Suffix                                object
dtype: object

In [10]:
df.to_csv('sources/judicial_emergencies.csv', date_format='%Y-%m-%d', index=False)